# 📘 Explorador y gestor de la base de datos Everest
Este notebook permite crear la base de datos, agregar usuarios y autenticar credenciales mediante funciones reutilizables.

In [ ]:
import sqlite3
import hashlib
from datetime import datetime
from typing import Optional
import pandas as pd
import yaml
from pathlib import Path


In [ ]:
def conectar_db(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        print(f"Conectado a la base de datos")
        return conn
    except sqlite3.Error as e:
        print(f"Error al conectar con la base de datos: {e}")
        return None


In [ ]:

def crear_tablas(DB_PATH):
    conn = conectar_db(DB_PATH)
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password TEXT NOT NULL,
        role TEXT NOT NULL
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS progress (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        level INTEGER,
        timestamp TEXT,
        FOREIGN KEY(user_id) REFERENCES users(id)
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS test_results (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        test_id TEXT,
        score INTEGER,
        timestamp TEXT,
        FOREIGN KEY(user_id) REFERENCES users(id)
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS avatar_profiles (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        skin TEXT,
        outfit TEXT,
        equipment TEXT,
        -- Columnas para las habilidades del avatar y su símil laboral
        fuerza INTEGER DEFAULT 0,
        agilidad INTEGER DEFAULT 0,
        equilibrio INTEGER DEFAULT 0,
        resistencia_clima INTEGER DEFAULT 0,
        vision INTEGER DEFAULT 0,
        agarre INTEGER DEFAULT 0,
        capacidad_carga INTEGER DEFAULT 0,
        habilidad_equipo INTEGER DEFAULT 0,
        comunicacion INTEGER DEFAULT 0,
        orientacion INTEGER DEFAULT 0,
        determinacion INTEGER DEFAULT 0,
        sentido_roca INTEGER DEFAULT 0,
        FOREIGN KEY(user_id) REFERENCES users(id)
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS medals (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        description TEXT,
        test_id TEXT,
        awarded_on TEXT,
        FOREIGN KEY(user_id) REFERENCES users(id)
    );
    """)

    conn.commit()
    conn.close()
    print("✅ Tablas creadas o ya existentes.")

# Ejemplo de uso (asegúrate de que DB_PATH apunte a tu base de datos)
if __name__ == "__main__":
    DB_PATH = 'your_database.db' # Cambia esto a la ruta de tu base de datos
    crear_tablas(DB_PATH)

In [ ]:
def agregar_usuario(DB_PATH,username: str, password: str, role: str = "player"):
    conn = conectar_db(DB_PATH)
    cursor = conn.cursor()
    hashed_pw = hashlib.sha256(password.encode()).hexdigest()

    try:
        cursor.execute("INSERT INTO users (username, password, role) VALUES (?, ?, ?)",
                       (username, hashed_pw, role))
        conn.commit()
        print(f"✅ Usuario '{username}' agregado con rol '{role}'")
    except sqlite3.IntegrityError:
        print(f"⚠️ El usuario '{username}' ya existe.")
    finally:
        conn.close()


In [ ]:
def autenticar_usuario(DB_PATH,username: str, password: str) -> Optional[dict]:
    conn = conectar_db(DB_PATH)
    cursor = conn.cursor()
    hashed_pw = hashlib.sha256(password.encode()).hexdigest()

    cursor.execute("SELECT id, role FROM users WHERE username = ? AND password = ?",
                   (username, hashed_pw))
    user = cursor.fetchone()
    conn.close()

    if user:
        return {"status": "success", "user_id": user[0], "role": user[1]}
    else:
        return None


In [ ]:

def listar_tablas(DB_PATH):
    conn = conectar_db(DB_PATH)
    if conn is None:
        raise RuntimeError("No se pudo conectar a la base de datos.")

    try:
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        tablas_df = pd.read_sql(query, conn)
        print("📋 Tablas encontradas:")
        # display(tablas_df)
        return tablas_df
    except Exception as e:
        print(f"❌ Error al listar tablas: {e}")
    finally:
        conn.close()

In [ ]:


def cargar_configuracion(ruta_config=None):
    """
    Carga un archivo config.yml. Si no se pasa ruta_config, se solicita por input.
    Retorna un diccionario con la configuración cargada.
    """
    if ruta_config is None:
        ruta_config = input("Ingrese la ruta del archivo config.yml: ").strip()

    ruta = Path(ruta_config)
    if not ruta.exists():
        raise FileNotFoundError(f"Archivo no encontrado: {ruta_config}")

    with open(ruta, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)

    print(f"Configuración cargada")
    return config


In [ ]:
def inicializar_perfiles_avatar_para_nuevos_usuarios(db_path):
    """
    Crea un perfil de avatar por defecto para cada usuario que no tenga uno en avatar_profiles.
    """
    conn = conectar_db(db_path)
    cursor = conn.cursor()

    cursor.execute("""
        INSERT INTO avatar_profiles (username)
        SELECT u.username
        FROM users u
        LEFT JOIN avatar_profiles a ON u.username = a.username
        WHERE a.username IS NULL
    """)
    conn.commit()
    conn.close()
    print("✅ Perfiles de avatar inicializados para usuarios sin perfil.")

In [ ]:



def actualizar_habilidad_avatar(db_path, user_id, habilidad, valor_cambio):
    """
    Actualiza el valor de una habilidad específica del avatar de un usuario.
    
    Args:
        db_path (str): Ruta al archivo de la base de datos.
        user_id (int): ID del usuario cuyo avatar se va a actualizar.
        habilidad (str): Nombre de la columna de la habilidad a actualizar (ej. 'fuerza_puntos').
        valor_cambio (int): Cantidad a sumar o restar al valor actual de la habilidad.
    
    Returns:
        bool: True si la actualización fue exitosa, False en caso contrario.
    """
    if not habilidad.endswith('_puntos'):
        print("⚠️ Nombre de habilidad inválido. Debe terminar en '_puntos'.")
        return False
        
    conn = conectar_db(db_path)
    cursor = conn.cursor()
    
    try:
        # Asegurarse de que el perfil exista antes de intentar actualizar
        cursor.execute("SELECT id FROM avatar_profiles WHERE user_id = ?", (user_id,))
        if cursor.fetchone() is None:
            print(f"⚠️ No existe un perfil de avatar para el usuario con ID {user_id}.")
            conn.close()
            return False

        # Prevenir inyección SQL asegurando que el nombre de la columna sea válido
        valid_habilidades = [
            'fuerza_puntos', 'agilidad_puntos', 'equilibrio_puntos',
            'resistencia_clima_puntos', 'vision_puntos', 'agarre_puntos',
            'capacidad_carga_puntos', 'habilidad_equipo_puntos',
            'comunicacion_puntos', 'orientacion_puntos', 'determinacion_puntos',
            'sentido_roca_puntos'
        ]
        if habilidad not in valid_habilidades:
            print(f"⚠️ La habilidad '{habilidad}' no es una columna válida para actualizar.")
            conn.close()
            return False

        # Actualizar la habilidad
        query = f"UPDATE avatar_profiles SET {habilidad} = {habilidad} + ? WHERE user_id = ?"
        cursor.execute(query, (valor_cambio, user_id))
        conn.commit()
        print(f"✅ Habilidad '{habilidad}' del usuario {user_id} actualizada por {valor_cambio}.")
        return True
    except sqlite3.Error as e:
        print(f"❌ Error al actualizar la habilidad: {e}")
        return False
    finally:
        conn.close()

In [ ]:


def obtener_perfil_avatar(db_path, user_id):
    """
    Obtiene el perfil completo de avatar para un usuario.
    """
    conn = conectar_db(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM avatar_profiles WHERE user_id = ?", (user_id,))
    perfil = cursor.fetchone()
    conn.close()
    
    if perfil:
        # Convertir las cadenas JSON de vuelta a listas
        # Asumiendo que el orden de las columnas es conocido para acceder por índice
        column_names = [description[0] for description in cursor.description]
        perfil_dict = dict(zip(column_names, perfil))
        
        if 'skin' in perfil_dict and perfil_dict['skin']:
            perfil_dict['skin'] = json.loads(perfil_dict['skin'])
        else:
            perfil_dict['skin'] = []
            
        if 'outfit' in perfil_dict and perfil_dict['outfit']:
            perfil_dict['outfit'] = json.loads(perfil_dict['outfit'])
        else:
            perfil_dict['outfit'] = []

        if 'equipment' in perfil_dict and perfil_dict['equipment']:
            perfil_dict['equipment'] = json.loads(perfil_dict['equipment'])
        else:
            perfil_dict['equipment'] = []
            
        return perfil_dict
    return None


In [ ]:
def crear_o_reemplazar_perfil_avatar(db_path, username, fuerza, agilidad, equilibrio, resistencia_frio, vision, agarre, capacidad_carga, habilidad_equipo,comunicacion, orientacion, determinacion, sentido_roca):
    """
    Crea un nuevo perfil de avatar para el usuario dado.
    Si ya existe, lo elimina y lo reemplaza con valores por defecto.
    """
    conn = conectar_db(db_path)
    cursor = conn.cursor()

    # Elimina el perfil si ya existe
    cursor.execute("DELETE FROM avatar_profiles WHERE username = ?", (username,))

    # Inserta un nuevo perfil con valores por defecto (por ejemplo: 50 en todo)
    cursor.execute("""
        INSERT INTO avatar_profiles (
            username, fuerza, agilidad, equilibrio, resistencia_frio,
            vision, agarre, capacidad_carga, habilidad_equipo,
            comunicacion, orientacion, determinacion, sentido_roca
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (username, fuerza, agilidad, equilibrio, resistencia_frio, vision, agarre, capacidad_carga, habilidad_equipo,comunicacion, orientacion, determinacion, sentido_roca))

    conn.commit()
    conn.close()
    print(f"✅ Perfil de avatar creado o reemplazado para '{username}'.")


---

## 🧪 Crear tablas y probar flujo completo

---

In [ ]:
'C:/Users/anmmu/OneDrive/Proyectos/PSYCH_CLIMBERS/PsychTest-Gaming-Hub/config.yml'
config = cargar_configuracion() 

In [ ]:

ENV_DIR = config["paths"]["env_dir"]
DB_PATH = ENV_DIR+ config["paths"]["db_file"]

In [ ]:
crear_tablas(DB_PATH)

In [ ]:
listar_tablas(DB_PATH)

In [ ]:

conn = sqlite3.connect(DB_PATH)
df_users = pd.read_sql("SELECT * FROM users", conn)
display(df_users)
conn.close()


In [ ]:
# Agregar un usuario de prueba
# agregar_usuario("demo_user", "1234", "player")

lista_usarios = [
    {'username' : "admin", 'password' : "987654321",'role' : "administrator"},
    {'username' : "Oscar", 'password' : "123456",'role' : "player"},
    {'username' : "Ana", 'password' : "123456",'role' : "player"},
    {'username' : "Andres", 'password' : "123456",'role' : "player"},
    {'username' : "Diana", 'password' : "123456",'role' : "player"},
    {'username' : "Maria", 'password' : "123456",'role' : "player"},
    {'username' : "sico_001", 'password' : "123456",'role' : "moderador"},
    {'username' : "sico_002", 'password' : "123456",'role' : "moderador"},
    {'username' : "invitado", 'password' : "123456",'role' : "player"},
    {'username' : "invitada", 'password' : "123456",'role' : "player"}
]
for usuario_nuevo in lista_usarios:
    agregar_usuario(DB_PATH,usuario_nuevo['username'], usuario_nuevo['password'], usuario_nuevo['role'])
    # print(usuario_nuevo['username'])
    print(usuario_nuevo)


In [ ]:

conn = sqlite3.connect(DB_PATH)
df_users = pd.read_sql("SELECT * FROM users", conn)
display(df_users)
conn.close()


In [ ]:
# 3. Inicializar perfiles de avatar para los usuarios existentes
inicializar_perfiles_avatar_para_nuevos_usuarios(DB_PATH)

In [41]:
tbl_sel = 'cuestionarios'
conn = sqlite3.connect(DB_PATH)
df_test = pd.read_sql(f"SELECT * FROM {tbl_sel}", conn)
print(f'table : {tbl_sel}')
display(df_test)
conn.close()

table : cuestionarios


,id,titulo,tematica,descripcion
0,1,Autoconocimiento Inicial,Psicología,Evalúa aspectos básicos de autoconocimiento en...
1,2,Resiliencia Emocional,Emociones,Cuestionario diseñado para identificar niveles...
2,3,Estilos de Liderazgo,Liderazgo,Explora el estilo predominante de liderazgo de...
3,4,Inteligencia Colaborativa,Trabajo en equipo,Evalúa habilidades para el trabajo colaborativ...
4,5,Toma de Decisiones Éticas,Ética y valores,Simula dilemas éticos donde se mide la capacid...


In [42]:
tbl_sel = 'preguntas'
conn = sqlite3.connect(DB_PATH)
df_test = pd.read_sql(f"SELECT * FROM {tbl_sel}", conn)
print(f'table : {tbl_sel}')
display(df_test)
conn.close()


table : preguntas


,id,cuestionario_id,texto,orden,tipo,pista,tiempo_limite,nivel_dificultad
0,1,1,¿Como te describirias en situaciones sociales?,2,seleccion,"No hay respuestas malas, solo preferencias per...",30,facil
1,2,1,¿Cómo te describirías en situaciones sociales?,2,seleccion,"No hay respuestas malas, solo preferencias per...",30,fácil
2,3,1,¿Qué tan seguido reflexionas sobre tus emociones?,3,seleccion,Piénsalo en términos semanales.,30,media


In [43]:
tbl_sel = 'respuestas'
conn = sqlite3.connect(DB_PATH)
df_test = pd.read_sql(f"SELECT * FROM {tbl_sel}", conn)
print(f'table : {tbl_sel}')
display(df_test)
conn.close()

table : respuestas


,id,pregunta_id,texto,es_correcta,etiqueta_lift
0,1,1,Busco una solución práctica de inmediato,true,decisión
1,2,1,Hablo con alguien para desahogarme,false,comunicación
2,3,1,Espero a ver si se resuelve solo,false,pasividad
3,4,1,Evito pensar en el problema,false,evasión
4,5,2,Me siento cómodo iniciando conversaciones,true,proactividad
5,6,2,Prefiero observar antes de participar,false,observación
6,7,2,Evito situaciones sociales grandes,false,introversión
7,8,2,Me adapto según el grupo,false,adaptabilidad
8,9,3,Casi todos los días,true,autoconciencia
9,10,3,Una vez a la semana,false,rutina


In [ ]:
crear_o_reemplazar_perfil_avatar(DB_PATH, "Oscar",50, 50, 50, 50, 100, 50, 50, 50, 50, 50, 50, 50)

In [ ]:
[
  {
    "table_name": "cuestionarios",
    "columns": [
      { "name": "id", "type": "INTEGER PRIMARY KEY AUTOINCREMENT" },
      { "name": "titulo", "type": "TEXT NOT NULL" },
      { "name": "tematica", "type": "TEXT" },
      { "name": "descripcion", "type": "TEXT" }
    ]
  },
  {
    "table_name": "preguntas",
    "columns": [
      { "name": "id", "type": "INTEGER PRIMARY KEY AUTOINCREMENT" },
      { "name": "cuestionario_id", "type": "INTEGER NOT NULL" },
      { "name": "texto", "type": "TEXT NOT NULL" },
      { "name": "orden", "type": "INTEGER" },
      { "name": "tipo", "type": "TEXT DEFAULT 'seleccion'" },
      { "name": "pista", "type": "TEXT" },
      { "name": "tiempo_limite", "type": "INTEGER DEFAULT 30" },
      { "name": "nivel_dificultad", "type": "TEXT DEFAULT 'media'" },
      { "name": "FOREIGN KEY(cuestionario_id)", "type": "REFERENCES cuestionarios(id)" }
    ]
  },
  {
    "table_name": "respuestas",
    "columns": [
      { "name": "id", "type": "INTEGER PRIMARY KEY AUTOINCREMENT" },
      { "name": "pregunta_id", "type": "INTEGER NOT NULL" },
      { "name": "texto", "type": "TEXT NOT NULL" },
      { "name": "es_correcta", "type": "BOOLEAN DEFAULT 0" },
      { "name": "etiqueta_lift", "type": "TEXT" },
      { "name": "FOREIGN KEY(pregunta_id)", "type": "REFERENCES preguntas(id)" }
    ]
  }
]


In [ ]:
[
  {
    "table": "preguntas",
    "data": {
      "cuestionario_id": 1,
      "texto": "¿Qué haces normalmente cuando enfrentas un problema inesperado?",
      "orden": 1,
      "tipo": "seleccion",
      "pista": "Piensa en tu reacción más frecuente",
      "tiempo_limite": 30,
      "nivel_dificultad": "media"
    }
  },
  {
    "table": "preguntas",
    "data": {
      "cuestionario_id": 1,
      "texto": "¿Cómo te describirías en situaciones sociales?",
      "orden": 2,
      "tipo": "seleccion",
      "pista": "No hay respuestas malas, solo preferencias personales.",
      "tiempo_limite": 30,
      "nivel_dificultad": "fácil"
    }
  },
  {
    "table": "preguntas",
    "data": {
      "cuestionario_id": 1,
      "texto": "¿Qué tan seguido reflexionas sobre tus emociones?",
      "orden": 3,
      "tipo": "seleccion",
      "pista": "Piénsalo en términos semanales.",
      "tiempo_limite": 30,
      "nivel_dificultad": "media"
    }
  }
]


In [ ]:
[
  {
    "table": "respuestas",
    "data": {
      "pregunta_id": 1,
      "texto": "Busco una solución práctica de inmediato",
      "es_correcta": true,
      "etiqueta_lift": "decisión"
    }
  },
  {
    "table": "respuestas",
    "data": {
      "pregunta_id": 1,
      "texto": "Evito pensar en el problema",
      "es_correcta": false,
      "etiqueta_lift": "evasión"
    }
  },
  {
    "table": "respuestas",
    "data": {
      "pregunta_id": 1,
      "texto": "Hablo con alguien para desahogarme",
      "es_correcta": false,
      "etiqueta_lift": "comunicación"
    }
  },
  {
    "table": "respuestas",
    "data": {
      "pregunta_id": 1,
      "texto": "Espero a ver si se resuelve solo",
      "es_correcta": false,
      "etiqueta_lift": "pasividad"
    }
  }
]


[
  {
    "table": "respuestas",
    "data": {
      "pregunta_id": 2,
      "texto": "Me siento cómodo iniciando conversaciones",
      "es_correcta": true,
      "etiqueta_lift": "proactividad"
    }
  },
  {
    "table": "respuestas",
    "data": {
      "pregunta_id": 2,
      "texto": "Prefiero observar antes de participar",
      "es_correcta": false,
      "etiqueta_lift": "observación"
    }
  },
  {
    "table": "respuestas",
    "data": {
      "pregunta_id": 2,
      "texto": "Evito situaciones sociales grandes",
      "es_correcta": false,
      "etiqueta_lift": "introversión"
    }
  },
  {
    "table": "respuestas",
    "data": {
      "pregunta_id": 2,
      "texto": "Me adapto según el grupo",
      "es_correcta": false,
      "etiqueta_lift": "adaptabilidad"
    }
  }
]


[
  {
    "table": "respuestas",
    "data": {
      "pregunta_id": 3,
      "texto": "Casi todos los días",
      "es_correcta": true,
      "etiqueta_lift": "autoconciencia"
    }
  },
  {
    "table": "respuestas",
    "data": {
      "pregunta_id": 3,
      "texto": "Una vez a la semana",
      "es_correcta": false,
      "etiqueta_lift": "rutina"
    }
  },
  {
    "table": "respuestas",
    "data": {
      "pregunta_id": 3,
      "texto": "Raramente lo hago",
      "es_correcta": false,
      "etiqueta_lift": "desconexión"
    }
  },
  {
    "table": "respuestas",
    "data": {
      "pregunta_id": 3,
      "texto": "Solo cuando algo me molesta",
      "es_correcta": false,
      "etiqueta_lift": "reactividad"
    }
  }
]
